The goal of this notebook is to 

First we need to set up a number of variables we going to use later in the notebook:
- OPENAI_API_KEY: to access the OpenAI embeddings (secret)
- AZURESEARCH_ADMIN_KEY: to access the Azure Search service (secret)
- AZURESEARCH_ENDPOINT (theoretically it's not a secret, but it's better to keep it in .env file)
- AZURESEARCH_INDEX_NAME (theoretically it's not a secret, but it's better to keep it in .env file)
- if you create index in Azure Portal (not programatically), the default field names will be different than the ones assumed by the AzureSearch in langchain_community package. You can set the following variables to match the field names in your index:
    - AZURESEARCH_FIELDS_CONTENT_VECTOR=text_vector (it's not a secret) 
    - AZURESEARCH_FIELDS_ID=chunk_id (it's not a secret)
    - AZURESEARCH_FIELDS_CONTENT=chunk (it's not a secret)
    
All above will be read from .env file.

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
_ = load_dotenv(find_dotenv(filename='.env'))

The first thing we need is to build a retriever. 


We will use Microsoft AI Search with hybrid mode to retrieve the most relevant documents for a given query.

Source code for langchain_community.vectorstores.azuresearch: 
https://api.python.langchain.com/en/latest/_modules/langchain_community/vectorstores/azuresearch.html#AzureSearch.similarity_search

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch

# OpenAI API data (for embeddings)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_version = "2023-05-15"
model = "text-embedding-ada-002"

# Azure AI Search data (for vector store)
vector_store_address = os.getenv("AZURESEARCH_ENDPOINT") 
vector_store_password = os.getenv("AZURESEARCH_ADMIN_KEY")
vector_store_index = os.getenv("AZURESEARCH_INDEX_NAME")

# Initialize the embeddings
embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key, 
    openai_api_version=openai_api_version, 
    model=model
)

# Alternatively, we may use AzureOpenAIEmbeddings
# from langchain_openai import AzureOpenAIEmbeddings
# embeddings = AzureOpenAIEmbeddings(
#     model="<embeding-model>",
#     azure_endpoint="<Azure_openai_endpoint>",
#     azure_ad_token_provider=token_provider
# )

# Initialize the vector store
vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=vector_store_index,
    embedding_function=embeddings.embed_query,
)

In [ ]:
# There multiple way of seting up the retriever, the issue here is to initialize it this way that is uses hybrid not only similarity search
# I use this one: https://stackoverflow.com/questions/78576496/hybrid-search-using-azure-ai-search-and-lang-chain-as-a-retriever
retriever = vector_store.as_retriever(k=2, search_type="hybrid")

# There is also Azure AI Search retriever https://python.langchain.com/v0.2/docs/integrations/retrievers/azure_ai_search/
# Here is the doc: https://python.langchain.com/v0.2/docs/integrations/retrievers/azure_ai_search/

# Another way discussed here: https://github.com/langchain-ai/langchain/discussions/18752

In [ ]:
# Some example queries
# query = "What are the fully local agents?"
# query = "What is Nvidia NIM API?"
query = "What is a generator function?"
docs = retriever.invoke(query)
for doc in docs:
    print(doc.page_content)

In [ ]:
# Let's define a main chain - it will use retrieved documents to generate an answer
# We also define two versions of the chain - one with Ollama (llama3.1) and one with OpenAI (gpt-4o-mini)

# PROVIDER = "ollama"
PROVIDER = "openai"

from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. 
    
    Analyze carefully and use the following documents to answer the user question. 
    
    If you don't know the answer, just say that you don't know. Do not make up an answer. 
    
    Keep your answer short and to the point.
    
    Question: {question} 
    Documents: {documents} 
    Answer: 
    """,
    input_variables=["question", "documents"],
)

if PROVIDER == "openai":
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
    )
else:
    llm = ChatOllama(
        model="llama3.1",
        temperature=0,
    )

main_chain = prompt | llm | StrOutputParser()

In [ ]:
# Define websearch tool - we use Tavilyn for this
from langchain.schema import Document
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults()